In [0]:
import pandas as pd
from collections import defaultdict

In [0]:
def get_parameter(key, default=""):
    try:
        return dbutils.widgets.get(key)
    except Exception as e:
        print('have no parameter "{}" in dbutils.widgets, return default "{}"'.format(key, default))
        return default

class VersionControl:
    def __init__(self, ws_type=None, version=None, comment=None):
        self.path = './save_path_log.txt'
        self.ws_type = ws_type
        self.version = version
        self.comment = comment
        
        if self.ws_type:
            self.check_if_version_exists()

    def check_if_version_exists(self, ws_type=None, version=None):
        ws_type = self.ws_type if not ws_type else ws_type
        version = self.version if not version else version
        ws_type_versions = self.find_versions(ws_type) 
        results = [x for x in ws_type_versions if x[0] == version]
        if len(results) > 0:
            print(results)
            return True
        else:
            print("the version has not been saved before")
            return False
    
    def save_version(self, ws_type=None, version=None, force=False):
        """
        force: if true则有一样的版本也存入，继续跑
        """
        self.ws_type = self.ws_type if not ws_type else ws_type
        self.version = self.version if not version else version
        check_status = self.check_if_version_exists()
        if check_status & ~force:
            raise ValueError('version already exists')
        else:
            with open(self.path, 'a', encoding='utf-8') as file:
                print(f'{self.ws_type}, {self.version}, {self.comment}')
                file.write(f'\n{self.ws_type}, {self.version}, {self.comment}')

    @staticmethod
    def get_all_versions():
        with open('./save_path_log.txt', 'r', encoding='utf-8') as file:
            versions = file.readlines()
        return versions
    
    def get_version_dict(self):
        version_info_list = self.get_all_versions()
        version_dict = defaultdict(list)
        for x in version_info_list:
            if x == '\n':
                pass
            else:
                version_dict[x.split(",")[0]].append((x.split(",")[1].strip(), x.split(",")[2].strip()))
        return version_dict
    
    def find_versions(self, ws_type=None):
        self.version_dict = self.get_version_dict()
        if not ws_type:
            return self.version_dict
        else:
            return self.version_dict.get(ws_type)

In [0]:
# finance_ds_inventory_dmt.finance_poc_datahub_mid_t1ws_wccs_str # t1.5 -> POC/T2 sales_type=1卖给T2，sales_type = 0 卖给POC 
# finance_ds_inventory_dmt.finance_poc_datahub_t1ws_wccs_str        # T1 -> 卖给T1.5/T2/POC
# 纯一批经销商的STR落表，sale_type =1是卖给售点，2是卖给2批，3是1.5批卖给售点的(从T1进货的部分)
# finance_ds_inventory_dmt.finance_poc_datahub_mid_ws_wccs_str     # T1 -> T1.5 payercode是T1.5的，filter sj_pay_to=T1
# finance_ds_inventory_dmt.finance_poc_datahub_wccs_str    # T1.5 -> poc 只看从ABI进货的


In [0]:
brand_family_scope_file_from_vivian = "/dbfs/FileStore/tables/srf/fifo_inv_simu/src_data/SAP_sku_CIO_BRANDFAMILY_Mapping_final.xlsx"
inv_data_path = '/dbfs/FileStore/tables/srf/fifo_inv_simu/src_data/进销存数据.xlsx'  # 期初库存
dec_inv_data_path = '/dbfs/FileStore/tables/srf/fifo_inv_simu/src_data/12月进销存数据.xlsx' # 12月的期末库存

In [0]:
def get_ws_name_payercode_mapping():
    mapping = spark.sql('select distinct int(wholesaler_pay_code) as payercode,  wholesaler_name as payername from budtech_brewdat_prod_ods.abi_cloud_wholesaler_ws_wholesaler').toPandas().set_index('payercode')
    return mapping

In [0]:
def get_sku_hl_mapping_table():
    sku_hl_mapping = spark.sql(
    """
    WITH CIO AS (
    SELECT
        DISTINCT cio_id AS cio_code,
        c_product as cio_name,
        regexp_replace(capacity, '([a-zA-Z])', '') / 1000.0 as container_L,
        -- trim(C2.Seg) AS Segment,
        packagesize
    FROM
        bees_products_ods.product_center_cio_sku C1
        LEFT JOIN bees_products_ods.product_center_cio_mapping C2 ON C2.cio_code = C1.cio_id
    WHERE
        deleted = 0
        AND regexp_replace(capacity, '([a-zA-Z])', '') > 0
        AND C2.Seg IS NOT NULL
        AND COALESCE(packagesize, 0) > 0
    )
    SELECT
    DISTINCT SC.sap_sku_code as SKUCode, K1.cio_code
    ,K1.cio_name
    ,K1.container_L * K1.packagesize/100 HL
    FROM
    CIO K1
    JOIN bees_products_ods.product_center_ref_sap_to_cio SC ON SC.cio_sku_code = K1.cio_code
    """
    ).toPandas()
    return sku_hl_mapping

def get_sku_sap_wccs_code_mapping_table():
    wccs_productid_sap_sku_mapping = spark.sql(
    """
    select distinct product_id, cast(sap_skudm as int) as SKUCode from wccs_ods.bas_material
    """).toPandas()
    return wccs_productid_sap_sku_mapping     

,SKUCode,cio_code,cio_name,HL
0,31942,100003,"HARBIN ICE哈尔滨冰纯BBT大瓶500ml , (12)",0.06
1,32197,100003,"HARBIN ICE哈尔滨冰纯BBT大瓶500ml , (12)",0.06
2,39474,100003,"HARBIN ICE哈尔滨冰纯BBT大瓶500ml , (12)",0.06
3,44458,100003,"HARBIN ICE哈尔滨冰纯BBT大瓶500ml , (12)",0.06
4,47386,100003,"HARBIN ICE哈尔滨冰纯BBT大瓶500ml , (12)",0.06
...,...,...,...,...
6542,106080,107032,"BOXING CAT BREWERY拳击猫低醇印度淡色艾尔KEG桶啤20000ml , (1)",0.20
6543,106218,107037,"BOXING CAT BREWERY拳击猫夏日酸啤KEG桶啤20000ml , (1)",0.20
6544,107111,107064,"GOOSE ISLAND鹅岛冬季深色艾尔CAN听装500ml , (12)",0.06
6545,107237,107066,"BLUE GIRL蓝妹铂金SBT铝瓶250ml , (24)",0.06


In [0]:
def get_used_brand_family():
    """
    used_wccs_sku_list: wccs product_id 列表，四个brand的
    sku_filtered是过滤了四个brand的sku mapping表；
    product_id_mapping是全量的sku hl mapping表
    """
    sku_brand_family_mapping = pd.read_excel(brand_family_scope_file_from_vivian) # src: Vivian
    used_brand_family = ['ISP', 'BUD', 'HBI', 'HBO', 'HKOW']
    # select distinct product_id, cast(sap_skudm as int) as SKUCode from wccs_ods.bas_material
    sku_hl_mapping = get_sku_hl_mapping_table()   # from Charlotte code
    product_id_mapping = get_sku_sap_wccs_code_mapping_table()
    used_brand_family = ['ISP', 'BUD', 'HBI', 'HBO', 'HKOW']
    used_brand_family_mapping = sku_brand_family_mapping[sku_brand_family_mapping['Brand Family'].isin(used_brand_family)].drop_duplicates(subset=['SKUCode'])[['SKUCode','Brand Family']]    
    # brand family data 匹配product id和百升数
    product_id_mapping = pd.merge(product_id_mapping, sku_hl_mapping, on='SKUCode', how='left')   # Pcode, SKUCode,HL
    used_brand_family_mapping = pd.merge(used_brand_family_mapping, product_id_mapping, on='SKUCode', how='left')
    used_brand_family_mapping['product_id'] = used_brand_family_mapping['product_id'].str.upper()
    used_wccs_sku_list = list(used_brand_family_mapping['product_id'].unique())
    return used_wccs_sku_list, used_brand_family_mapping, product_id_mapping

In [0]:
def get_inv_data(used_wccs_sku_list, sku_filtered, used_qty_col='期初库存', path=inv_data_path):
    """
        箱数_col：期初库存
        hl_col: inv_hl     
    """
    # # begin_inv = pd.read_csv('src_data/库存余额表DB-202212.csv')       # 2022年末的期末库存就是2023年的期初库存
    begin_inv = pd.read_excel(path)
    begin_inv['物料代码'] = begin_inv['物料代码'].str.upper()
    begin_inv = begin_inv[begin_inv['物料代码'].isin(used_wccs_sku_list)]
    sku_filtered_info = sku_filtered.sort_values(by='product_id').drop_duplicates(subset=['product_id'])
    sku_filtered_info = sku_filtered_info[sku_filtered_info['product_id'].notnull()]
    begin_inv = pd.merge(begin_inv, sku_filtered_info[['product_id', 'SKUCode', 'Brand Family', 'HL']], left_on='物料代码',right_on='product_id', how='left')
    unfound_sku = begin_inv[begin_inv['SKUCode'].isnull()]
    if len(unfound_sku) > 0:
        print("没对应上SKUCode/HL的wccs product id", unfound_sku['product_id'].unique())
    begin_inv['inv_hl'] = begin_inv[used_qty_col] * begin_inv['HL']    # 转换成百升
    return begin_inv

In [0]:
def get_stw_data(start=202101, end=202410):
    """
        箱数_col：stw_qty
        hl_col: stw_hl
        month_col: billingyearmonth
    """
    # # stw = pd.read_excel('src_data/stw_str_241118.xlsx', sheet_name='STW')        # 2023.1.1~2023.12.31
    stw = spark.sql('select * from finance_ds_inventory_dmt.info_order_database_finance_stw').toPandas()          # 2021.1~2024.10
    stw.columns = [x.lower() for x in stw.columns]
    stw = stw[(stw['billingyearmonth'] >= start) & (stw['billingyearmonth'] <= end)]
    return stw

def get_str_data(sku_filtered, str_db_path, start=202101, end=202410):
    """
        sku_filtered：skucode and hl, for hl mapping
        str_db_path: 如果是字符串直接读表，或者直接传入聚合好的数
        month_col: month
    """
    # # ws_str = pd.read_excel('src_data/stw_str_241118.xlsx', sheet_name='STR')
    if isinstance(str_db_path, str):
        ws_str = spark.sql(f"select * from {str_db_path}").toPandas()
    elif isinstance(str_db_path, pd.DataFrame):
        ws_str = str_db_path
    else:
        ws_str = str_db_path.toPandas()
    # ws_str = spark.sql('select * from finance_ds_inventory_dmt.finance_poc_datahub_wccs_str').toPandas()       # 2021.1~2024.11
    ws_str['payercode'] = ws_str['payercode'].astype(int)
    ws_str = ws_str[(ws_str['month'] >= start) & (ws_str['month'] <= end)]
    ws_str.columns = [x.lower() for x in ws_str.columns]
    sku_filtered_info = sku_filtered.sort_values(by='SKUCode').drop_duplicates(subset=['SKUCode'])
    ws_str = pd.merge(ws_str, sku_filtered_info[['SKUCode', 'HL']], left_on='skucode', right_on='SKUCode', how='left')
    unfound_sku = ws_str[ws_str['HL'].isnull()]
    if len(unfound_sku) > 0:
        print(unfound_sku)
    ws_str['str_hl'] = ws_str['str_qty'] * ws_str['HL']
    return ws_str

In [0]:
def cat_code(df):
    return str(df['t1_code']) + "_" + str(df['t15_code'])

def split_paycode(df):
    v = df['payercode'].split("_")
    return v[0], v[1]

In [0]:
def get_all_t1_t15_comb():
    """
    所有t1 * t15 的组合
    """
    df = spark.sql(f'''
        select payercode as t15_code, sj_pay_to as t1_code  
        from finance_ds_inventory_dmt.finance_poc_datahub_mid_ws_wccs_str 
        where deleted = 0  
        group by all 
        ''')
    print(df.count()) # 1878
    df = df.toPandas()
    df['t15_code'] = df['t15_code'].apply(lambda x: str(int(x)))
    df['t1_code'] = df['t1_code'].apply(lambda x: str(int(x)))

    return df
# get_all_t1_t15_comb()

def get_t15_stw_monthly_from_t1(sku_filtered, start='202101', end='202410'):
    """
    t15 每个月从t1 的进货量

    STR T1 to T1.5
    payercode =T1.5 A, sj_pay_to = T1
    payercode =T1.5 B, sj_pay_to = T1
    """
    print(start, end)
    df = spark.sql(f'''
        select  payercode as t15_code, sj_pay_to as t1_code, Brand_Family_ComDB, wccs_sku_name, skucode, month as billingyearmonth, sum(str_qty) as str_qty 
        , payercode, payer_bu_name, payer_region_name
        from finance_ds_inventory_dmt.finance_poc_datahub_mid_ws_wccs_str 
        where deleted = 0 
        group by all  
        ''')
    stw = df.toPandas()
    #TODO payercode t1,t15 组合？
    stw['t1_code'] = stw['t1_code'].apply(lambda x: str(int(x)))
    stw['t15_code'] = stw['t15_code'].apply(lambda x: str(int(x)))
    stw = stw[(stw['billingyearmonth'] >= start) & (stw['billingyearmonth'] <= end)]
    stw.columns = [x.lower() for x in stw.columns]
    sku_filtered_info = sku_filtered.sort_values(by='SKUCode').drop_duplicates(subset=['SKUCode'])
    stw = pd.merge(stw, sku_filtered_info[['SKUCode', 'HL']], left_on='skucode', right_on='SKUCode', how='left')
    unfound_sku = stw[stw['HL'].isnull()]
    if len(unfound_sku) > 0:
        print(unfound_sku)
    stw['stw_hl'] = stw['str_qty'] * stw['HL']
    stw['payercode']  = stw.apply(lambda x : cat_code(x), axis=1)
    return stw


def get_t15_inv(month='202012'):
    pass


1878


In [0]:
def get_t15_str_monthly(sku_filtered, start='202101', end='202410'):
    """
    t15 的str量
    记到不同的t1身上

    T1.5 to POC
    sale type = 3, sub_payercode = T1.5 A
    sale type = 3, sub_payercode = T1.5 B

    """
    print(start, end)
    df = spark.sql(f'''
        select payercode as t1_code, sub_payercode as t15_code,  Brand_Family_ComDB, wccs_sku_name, skucode,  month, sum(str_qty) as str_qty , 
        sub_payercode as payercode, payer_bu_name, payer_region_name
        from finance_ds_inventory_dmt.finance_poc_datahub_t1ws_wccs_str 
        where deleted = 0
        and sale_type = 3
        group by all  
        ''')
    ws_str = df.toPandas()
    ws_str['t1_code'] = ws_str['t1_code'].apply(lambda x: str(int(x)))
    ws_str['t15_code'] = ws_str['t15_code'].apply(lambda x: str(int(x)))
    ws_str = ws_str[(ws_str['month'] >= start) & (ws_str['month'] <= end)]
    ws_str.columns = [x.lower() for x in ws_str.columns]
    sku_filtered_info = sku_filtered.sort_values(by='SKUCode').drop_duplicates(subset=['SKUCode'])
    ws_str = pd.merge(ws_str, sku_filtered_info[['SKUCode', 'HL']], left_on='skucode', right_on='SKUCode', how='left')
    unfound_sku = ws_str[ws_str['HL'].isnull()]
    if len(unfound_sku) > 0:
        print(unfound_sku)
    ws_str['str_hl'] = ws_str['str_qty'] * ws_str['HL']
    ws_str['payercode']  = ws_str.apply(lambda x : cat_code(x), axis=1)
    return ws_str

In [0]:
# %sql
# -- select * from finance_ds_inventory_dmt.finance_poc_datahub_t1ws_wccs_str limit 10

payercode,payer_name,payer_bu_code,payer_bu_name,payer_region_code,payer_region_name,sub_payercode,sub_payer_name,month,Brand_Family_ComDB,wccs_sku_name,sale_type,skucode,str_qty,dmt_ins_ts,dmt_upd_ts,deleted
0030017642,福建瑞福祥实业有限责任公司,0000100004,东南区|BU_SE,0000105646,福建泉州|Region_Fujian_Quanzhou,0030017642,福建瑞福祥实业有限责任公司,202304,BUD,百威9.7度330ML1X24纸箱听装,1,25247,9954,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0
0030014908,大连虹拓贸易有限公司,0000102688,东北|BU_NE,0000105939,辽西南区|Region_Liaoning_Southwest,0030014908,大连虹拓贸易有限公司,202104,HBO,哈尔滨醇爽10度500ML1X12 塑包听装,1,50276,9594,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0
0030019043,汕头市潮联优品贸易有限公司,0000104652,南区|BU_S,0000105639,广东东区|Region_Guangdong_East,0030019043,汕头市潮联优品贸易有限公司,202110,HKOW,哈尔滨小麦王10度330ML4X6纸箱听装(塑膜内包),1,55971,59000,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0
0030016392,梅州市金乐丰贸易有限公司,0000104652,南区|BU_S,0000105639,广东东区|Region_Guangdong_East,0030016392,梅州市金乐丰贸易有限公司,202204,BUD,百威9.7度330ML2X9塑包听装-促销版A,1,91791,5865,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0
0030011177,广昌县鹏兴商行,0000100004,东南区|BU_SE,0000105990,江西东部|Region_Jiangxi_East,0030011177,广昌县鹏兴商行,202105,BUD,百威9.7度330ML1X24纸箱OW瓶装,1,25141,2693,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0
0030016321,天元区联达商行,0000100004,东南区|BU_SE,0000105135,湖南|Region_Hunan,0030016321,天元区联达商行,202312,HBI,哈尔滨冰纯9.1度500ML1X12纸箱听装（带提手）-促销版A,1,87855,2000,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0
0030016392,梅州市金乐丰贸易有限公司,0000104652,南区|BU_S,0000105639,广东东区|Region_Guangdong_East,0030016392,梅州市金乐丰贸易有限公司,202105,BUD,百威9.7度500ML1X12纸箱回瓶装(广东特供),1,87711,44062,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0
0030008740,科尔沁右翼前旗乾润商贸有限责任公司,0000102688,东北|BU_NE,0000106041,吉林西区|Region_Jilin_West,0030008740,科尔沁右翼前旗乾润商贸有限责任公司,202206,HBI,哈尔滨冰纯9.1度500ML1X12纸箱回瓶装-促销版D,1,43416,128937,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0
0030002987,重庆高美食品酒业有限责任公司,0000104662,中区|BU_C,0000105122,重庆市|Region_Chongqing,0030002987,重庆高美食品酒业有限责任公司,202306,HBO,哈尔滨清爽8度330ML1X24纸箱回瓶装(无格挡)-白瓶,1,71301,4225,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0
0030002987,重庆高美食品酒业有限责任公司,0000104662,中区|BU_C,0000105122,重庆市|Region_Chongqing,0030002987,重庆高美食品酒业有限责任公司,202405,BUD,CT_百威9.7度330ML1X24纸箱OW铝瓶装(C2C),1,67235,10149,2024-12-19T18:35:21.935Z,2024-12-19T18:35:21.935Z,0


In [0]:
# %sql
# select * from finance_ds_inventory_dmt.info_order_database_finance_stw limit 10

SoldToCode,SoldToName,PayerCode,PayerName,payer_bu_code,payer_bu_name,payer_region_code,payer_region_name,BillingYearMonth,Brand_Family_ComDB,SKUCode,SKUName,stw_hl,stw_qty,dmt_ins_ts,dmt_upd_ts,deleted
10047076,鄂伦春自治旗大杨树镇华奕佳丰酒业,30019511,鄂伦春自治旗大杨树镇华奕佳丰酒业,0000102688,东北|BU_NE,0000105627,黑龙江东区|Region_Heilongjiang_East,202311,BUD,100945,百威9.7度460ML1X12纸箱回瓶装-钻石标-促销版A,132.15,2394,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
10032225,广州韶记贸易有限公司,30016284,广州韶记贸易有限公司,0000104652,南区|BU_S,0000105642,广东大广州|Region_Guangdong_Greater Guan,202311,BUD,37995,百威纯生8度500ML1X12纸箱回瓶装(无格挡),45.36,756,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
10027311,海南明峰贸易有限公司,30014760,海南明峰贸易有限公司,0000104652,南区|BU_S,0000105644,海南|Region_Hainan,202311,ISP,76353,福佳白11.7度275ML1X24纸箱OW瓶装,12.01,182,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
10021938,北京京东世纪贸易有限公司(广州DC),30011727,北京京东世纪信息技术有限公司（广州DC）,0000104652,南区|BU_S,0000105642,广东大广州|Region_Guangdong_Greater Guan,202311,ISP,82204,三宝乐11度350ML1X24纸箱听装,29.40,350,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
10043304,成都红照天商贸有限公司(宏鑫),30015767,成都红照天商贸有限公司,0000104649,西区|BU_W,0000105637,四川成都|Region_Sichuan_Chengdu,202311,BUD,63259,百威9.7度500ML1X12纸箱听装-促销版A(带提手),43.20,720,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
10036493,凤城市金叶商贸有限公司,30017693,凤城市金叶商贸有限公司,0000102688,东北|BU_NE,0000105630,辽宁北区|Region_Liaoning_North,202311,HBO,25503,哈尔滨冰爽8度330ML1X24纸箱回瓶装,37.07,468,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
10044345,泸州市龙马潭区黄前彩副食经营部(叙永分仓),30019052,泸州市龙马潭区黄前彩副食经营部,0000104649,西区|BU_W,0000105635,四川南区|Region_Sichuan_South,202310,HBO,91504,哈尔滨清爽8度500ML1X12纸箱回瓶装-新VI版,64.80,1080,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
10046689,绥芬河市禹翔百威啤酒经销处,30019425,绥芬河市禹翔百威啤酒经销处,0000102688,东北|BU_NE,0000105780,黑龙江南区|Region_Heilongjiang_South,202312,BUD,100945,百威9.7度460ML1X12纸箱回瓶装-钻石标-促销版A,111.28,2016,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
10027839,广州吉瀚丰翔商贸有限公司,30014955,广州吉瀚丰翔商贸有限公司,0000104652,南区|BU_S,0000105642,广东大广州|Region_Guangdong_Greater Guan,202311,BUD,95798,百威昕蓝7.5度500ML1X12纸箱听装-常规版,442.26,7371,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
10054156,望奎县君威食品有限公司,30020449,望奎县君威食品有限公司,0000102688,东北|BU_NE,0000105627,黑龙江东区|Region_Heilongjiang_East,202311,BUD,25162,百威纯生8度500ML1X12纸箱回瓶装,233.28,3888,2024-12-16T08:01:25.961Z,2024-12-16T08:01:25.961Z,0
